In [1]:
#@title

%%capture
!pip install transformers
!pip install gradio
import gradio as gr
import nltk
nltk.download('punkt')
from tabulate import tabulate
tabulate.PRESERVE_WHITESPACE = True
from fincat_utils import extract_context_words
from fincat_utils import bert_embedding_extract
import pickle
lr_clf = pickle.load(open("lr_clf_FiNCAT.pickle",'rb'))

In [2]:
#@title
#txt = 'In the year 2021 the markets were bullish. We expect to boost our sales by 80% this quarter.'
def score_fincat(txt):
  li = []
  for word in txt.split():
    if any(char.isdigit() for char in word):
      st = txt.index(word)
      ed = st + len(word)
      x = {'paragraph' : txt, 'offset_start':st, 'offset_end':ed}
      context_text = extract_context_words(x)
      features = bert_embedding_extract(context_text, word)
      prediction = lr_clf.predict(features.reshape(1, 768))
      prediction_probability = '{:.4f}'.format(round(lr_clf.predict_proba(features.reshape(1, 768))[:,1][0], 4))
      
      li.append([word,'    In-claim' if prediction==1 else 'Out-of-Claim', prediction_probability])
  headers = ['numeral', 'prediction', 'probability']
  return tabulate(li, headers, tablefmt="plain", floatfmt=".4f")

iface = gr.Interface(fn=score_fincat, inputs="textbox", title="FiNCAT",description="Financial Numeral Claim Analysis Tool", outputs="textbox", allow_flagging="never")
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://59019.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7fe42680aa10>,
 'http://127.0.0.1:7860/',
 'https://59019.gradio.app')